# Simplest 1D Gan
Simplest example based on this tutorial:
* [Blog post](http://blog.aylien.com/introduction-generative-adversarial-networks-code-tensorflow/)
* [Code](https://github.com/AYLIEN/gan-intro)
* [Reference of the reference](http://blog.evjang.com/2016/06/generative-adversarial-nets-in.html)

### Other References
* [Fantastic GANS and Where to find them](http://guimperarnau.com/blog/2017/03/Fantastic-GANs-and-where-to-find-them)
* [Karpathy Demo](http://cs.stanford.edu/people/karpathy/gan/)
* [Probabiliy Theory Basics](https://medium.com/@dubovikov.kirill/probabiliy-theory-basics-4ef523ae0820)
* [Ian Goodfellow Paper](https://arxiv.org/pdf/1406.2661.pdf)
* [How do GANs intuitively work](https://hackernoon.com/how-do-gans-intuitively-work-2dda07f247a1)
* [Mode Collapse](http://aiden.nibali.org/blog/2017-01-18-mode-collapse-gans/)
* [Gan Objective](http://aiden.nibali.org/blog/2016-12-21-gan-objective/)
* [Tensorflow sharing variables](https://www.tensorflow.org/programmers_guide/variable_scope)
* [BEGAN blog](https://blog.heuritech.com/2017/04/11/began-state-of-the-art-generation-of-faces-with-generative-adversarial-networks/)
* [BEGAN paper](https://arxiv.org/pdf/1703.10717.pdf)
* [BEGAN Tensorflow](https://github.com/carpedm20/BEGAN-tensorflow)
* [Veegan blog](https://akashgit.github.io/VEEGAN/)
* [Veegan paper](https://arxiv.org/pdf/1705.07761.pdf)
* [Unrolled Gans](https://arxiv.org/pdf/1611.02163.pdf)
* [F-Gan](https://arxiv.org/pdf/1606.00709.pdf)
* [Gans in Keras](https://github.com/eriklindernoren/Keras-GAN)

In [1]:
import numpy as np
import tensorflow as tf

# Include modules from other directories
import sys
sys.path.append('../tensorflow/')
import model_util as util
import anim_util as anim

import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(0)

# Fix seed to reproduce same results
seed = 42
np.random.seed(seed)
tf.set_random_seed(seed)

# Some meta parameters
HIDDEN_SIZE=4
start_lr = 0.005
decay = 0.95
num_steps = 5000
batch_size = 8
num_decay_steps = 150
logs_path = './logs'
save_dir = './save'
gpu_fraction = 0.1

# Delete logs directory if exist
if os.path.exists(logs_path):        
    os.system("rm -rf " + logs_path)

In [2]:
class DataDistribution(object):
    def __init__(self):
        self.mu = 4
        self.sigma = 0.5

    def sample(self, N):
        samples = np.random.normal(self.mu, self.sigma, N)
        samples.sort()
        return samples


class GeneratorDistribution(object):
    def __init__(self, range):
        self.range = range

    def sample(self, N):
        return np.linspace(-self.range, self.range, N) + np.random.random(N) * 0.01    

### Create functions for Generator and Discriminator
Observe that the discriminator on this example is more powerfull than the generator, if the mini-batch feature is not used.

In [3]:
# Minibatch feature
# https://arxiv.org/pdf/1606.03498.pdf
def minibatch(input, num_kernels=5, kernel_dim=3):
    # Transform feature into a matrix
    x = util.linear_std(input, num_kernels * kernel_dim, name='minibatch', stddev=0.02)
    activation = tf.reshape(x, (-1, num_kernels, kernel_dim))        
    
    # Calculate the L1 and then the sum of the negative exponential
    diffs = tf.expand_dims(activation, 3) - tf.expand_dims(tf.transpose(activation, [1, 2, 0]), 0)
    abs_diffs = tf.reduce_sum(tf.abs(diffs), 2)
    minibatch_features = tf.reduce_sum(tf.exp(-abs_diffs), 2)
    
    # Concatenate results back on the input (of some layer of the discriminator)
    return tf.concat([input, minibatch_features], 1)    

def generator(input, hidden_size):
    h0 = tf.nn.relu(util.linear_std(input, hidden_size, 'g0'))    
    # Here we cannot use a tanh because our data has mean bigger than zero (Just this case)
    h1 = util.linear_std(h0, 1, 'g1')
    return h1

def discriminator(input, hidden_size):
    h0 = util.lrelu(util.linear_std(input, hidden_size * 2, 'd0'))
    h1 = util.lrelu(util.linear_std(h0, hidden_size * 2, 'd1'))
    h2 = util.lrelu(util.linear_std(h1, hidden_size * 2, 'd2'))
    h3 = tf.sigmoid(util.linear_std(h2, 1, 'd3'))
    return h3

def discriminator_mb(input, hidden_size):
    h0 = util.lrelu(util.linear_std(input, hidden_size * 2, 'd0'))
    h1 = util.lrelu(util.linear_std(h0, hidden_size * 2, 'd1'))
    # Notice that using the mini-batch technique the discriminator can be smaller
    h2 = minibatch(h1)    
    h3 = tf.sigmoid(util.linear_std(h2, 1, 'd3'))
    return h3

### Define GAN model

In [4]:
with tf.variable_scope('GAN'):
    # z is the generator vector input
    z = tf.placeholder(tf.float32, shape=(None, 1), name='Z')
    # x is the placeholder for real data
    x = tf.placeholder(tf.float32, shape=(None, 1), name='X')
    
    with tf.variable_scope('G'):        
        # G will have the generator output tensor
        G = generator(z, HIDDEN_SIZE)

    with tf.variable_scope('D') as scope:        
        #D_real = discriminator(x, HIDDEN_SIZE)
        D_real = discriminator_mb(x, HIDDEN_SIZE)
        # Make discriminator for real/fake input share the same set of weights
        scope.reuse_variables()
        #D_fake = discriminator(G, HIDDEN_SIZE)
        D_fake = discriminator_mb(G, HIDDEN_SIZE)

### Define Loss function

In [5]:
with tf.variable_scope('loss_disc'):
    # Define losses
    loss_d = tf.reduce_mean(-tf.log(D_real) - tf.log(1 - D_fake))

with tf.variable_scope('loss_gen'):
    loss_g = tf.reduce_mean(-tf.log(D_fake))

### Get parameters from Generator and Discriminator

In [6]:
vars = tf.trainable_variables()
d_params = [v for v in vars if v.name.startswith('GAN/D/')]
g_params = [v for v in vars if v.name.startswith('GAN/G/')]

### Create the Session
Basically ask tensorflow to build the graph

In [7]:
# Avoid allocating the whole memory
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

### Define the solver
We want to use the Adam solver to minimize or loss function.

In [8]:
def optimizer(loss, var_list, name='Solver'):
    # Solver configuration
    # Get ops to update moving_mean and moving_variance from batch_norm
    # Reference: https://www.tensorflow.org/api_docs/python/tf/contrib/layers/batch_norm
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.name_scope(name):
        global_step = tf.Variable(0, trainable=False)
        starter_learning_rate = start_lr
        # decay every 10000 steps with a base of 0.96
        learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                                   num_decay_steps, decay, staircase=True)

        # Basically update the batch_norm moving averages before the training step
        # http://ruishu.io/2016/12/27/batchnorm/
        with tf.control_dependencies(update_ops):
            train_step = tf.train.AdamOptimizer(
                learning_rate).minimize(loss, global_step=global_step, var_list=var_list)
    
    return train_step, learning_rate


In [9]:
opt_disc, lr_disc = optimizer(loss_d, d_params, 'Solver_Disc')
opt_gen, lr_gen = optimizer(loss_g, g_params, 'Solver_Gen')

### Add some variables to tensorboard

In [10]:
# Create histogram for labels
tf.summary.histogram("data_dist", x)
tf.summary.histogram("latent", z)
tf.summary.histogram("generator", G)

# Monitor loss, learning_rate, global_step, etc...
tf.summary.scalar("loss_disc", loss_d)
tf.summary.scalar("loss_gen", loss_g)
tf.summary.scalar("lr_disc", lr_disc)
tf.summary.scalar("lr_gen", lr_gen)
# merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

# Configure where to save the logs for tensorboard
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

### Initialize the values (Random values of weights)

In [11]:
# Initialize all random variables (Weights/Bias)
sess.run(tf.global_variables_initializer())

In [ ]:
data = DataDistribution()
gen = GeneratorDistribution(range=8)

In [ ]:
anim_frames = []
for step in range(num_steps):
    # Gather some real data and some latent z values
    x_np = data.sample(batch_size)
    z_np = gen.sample(batch_size)    
    
    # update discriminator
    sess.run([loss_d, opt_disc], {x: np.reshape(x_np, (batch_size, 1)),z: np.reshape(z_np, (batch_size, 1))})

    # update generator
    z_np = gen.sample(batch_size)    
    sess.run([loss_g, opt_gen], {z: np.reshape(z_np, (batch_size, 1))})
    
    # write logs at every iteration
    summary = merged_summary_op.eval(
        feed_dict={x: np.reshape(x_np, (batch_size, 1)),z: np.reshape(z_np, (batch_size, 1))})
    summary_writer.add_summary(summary, step)
    
    # Handle animation (Make the whole training slow ...)
    anim_frames.append(anim.samples(D_real, G, x, z, sess, data, gen.range, batch_size))

### Create a video
Create a video with every iteration displaying:
* Real data
* Decision boundary
* Generated Data

In [ ]:
anim.save_animation(anim_frames, './plot.mp4', gen.range)